In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import pandas as pd
import numpy as np
import h5py
import joblib

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline

# Data Processing



In [2]:
def load_data(filename):
    """
    이미지를 불러옵니다.
    Parameters:
        filename: str
            h5 파일에서 데이터를 불러옵니다.
    Returns:
        np.ndarray, pd.DataFrame, np.ndarray, 
        train 이미지, train spot 정보, test 이미지, test spot 정보
    """
    images, images_test = list(), list()
    spots, spots_test = list(), list()
    with h5py.File(filename, "r") as h5file:
        train_images = h5file["images/Train"]
        train_spots = h5file["spots/Train"]
    
        num_train_slides = len(train_images)
        # Train 이미지를 불러옵니다.
        # 하나의 텐서로 만들기 위해 이미지의 크기를 2000x2000으로 균일하게 만듭니다.
        for i, slide_name in enumerate(train_images.keys()):
            image = np.array(train_images[slide_name])
            p1 = 2000 - image.shape[0]
            p2 = 2000 - image.shape[1]
            images.append(
                np.pad(image, [(0, p1), (0, p2), (0, 0)], 'edge')
            )
            spots.append(pd.DataFrame(np.array(train_spots[slide_name])).assign(slide = i))
        # Test 이미지를 불러옵니다.
        test_images = h5file["images/Test"]
        test_spots = h5file["spots/Test"]
        sample = 'S_7'
        image = np.array(test_images[sample])
        p1 = 2000 - image.shape[0]
        p2 = 2000 - image.shape[1]
        images_test.append(np.pad(image, [(0, p1), (0, p2), (0, 0)], 'edge'))
        spots_test.append(pd.DataFrame(np.array(test_spots[sample])).assign(slide = 0))
    # EfficientNet의 형식으로 바꿉니다.
    with tf.device('/CPU:0'):
        images = tf.constant(tf.keras.applications.efficientnet.preprocess_input(images))
    df_spots = pd.concat(spots)
    with tf.device('/CPU:0'):
        images_test = tf.constant(tf.keras.applications.efficientnet.preprocess_input(images_test))
    df_spots_test = pd.concat(spots_test)
    return images, df_spots, images_test, df_spots_test

def make_img_proc_info(df, img_with, img_height):
    """
    
    """
    return df.assign(
        left = lambda x: x['x'] - img_width // 2,
        right = lambda x: x['x'] + img_width // 2,
        top = lambda x: x['y'] - img_height // 2,
        bottom = lambda x: x['y'] + img_height // 2,
        lpad = lambda x: -(x['left'].where(x['left'] < 0, 0)),
        rpad = lambda x: -(2000 - x['right']).where(x['right'] > 2000, 0),
        tpad = lambda x: -(x['top'].where(x['top'] < 0, 0)),
        bpad = lambda x: -(2000 - x['bottom']).where(x['bottom'] > 2000, 0)
    ).assign(
        left = lambda x: x['left'].clip(0, 2000),
        right = lambda x: x['right'].clip(0, 2000),
        top = lambda x: x['top'].clip(0, 2000),
        bottom = lambda x: x['bottom'].clip(0, 2000),
    )

def create_tf_ds(df):
    if (pd.Series(targets).isin(df.columns)).all():
        return tf.data.Dataset.from_tensor_slices(
            ({
                i: df[i] for i in ['left', 'right', 'top', 'bottom', 'slide', 'lpad', 'rpad', 'tpad', 'bpad']
            }, df[targets])
        )
    else:
        return tf.data.Dataset.from_tensor_slices({
            i: df[i] for i in ['left', 'right', 'top', 'bottom', 'slide', 'lpad', 'rpad', 'tpad', 'bpad']
        })

def proc_images(X, images):
    return tf.pad(
        images[X['slide'], X['left']:X['right'], X['top']:X['bottom'], :], 
        paddings = [(X['lpad'], X['rpad']), (X['tpad'], X['bpad']), (0, 0)],
        constant_values=1
    )

augmentation_layers = [
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
]

def data_augmentation(x):
    for layer in augmentation_layers:
        x = layer(x)
    return x

images, df_spots, images_test, df_spots_test = load_data("data/elucidata_ai_challenge_data.h5")
targets = [i for i in df_spots.columns if i.startswith('C')]

target_proc = FunctionTransformer(np.log, np.exp)
target_proc.fit(df_spots[targets])
df_spots[targets] = target_proc.transform(df_spots[targets])

In [3]:
img_width = 224
img_height = 224

df_spots = make_img_proc_info(df_spots, img_width, img_height)
df_spots_test = make_img_proc_info(df_spots_test, img_width, img_height)

In [4]:
input_shape = (img_width, img_height, 3)
enet = tf.keras.applications.EfficientNetB0(
    include_top = False, 
    weights = 'imagenet',
    input_shape = input_shape,
    pooling = 'avg'
)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [5]:
df_spots['slide'].unique()

array([0, 1, 2, 3, 4, 5])

In [6]:
from tqdm.notebook import tqdm
class TqdmEpochProgress(tf.keras.callbacks.Callback):
    def __init__(self, epochs):
        super().__init__()
        self.epochs = epochs
        self.progress_bar = None

    def on_train_begin(self, logs=None):
        self.progress_bar = tqdm(total=self.epochs, desc="Epochs")

    def on_epoch_end(self, epoch, logs=None):
        log_str = f"loss: {logs.get('loss'):.4f}"
        if 'val_loss' in logs:
            log_str += f", val_loss: {logs.get('val_loss'):.4f}"
        self.progress_bar.set_postfix_str(log_str)
        self.progress_bar.update(1)

    def on_train_end(self, logs=None):
        self.progress_bar.close()

# Validation

In [7]:
batch_size = 32
ds_cv_train = create_tf_ds(
    df_spots.loc[df_spots['slide'] != 5].pipe(
        lambda x: pd.concat([
            x, x.sample(n = batch_size - (len(x) % batch_size))
        ])
    )
).shuffle(5000).map(
    lambda X, Y: (proc_images(X, images), Y)
).map(
    lambda X, Y: (data_augmentation(X), Y)
).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

ds_valid = create_tf_ds(df_spots.loc[df_spots['slide'] == 5]).map(
    lambda X, Y: (proc_images(X, images), Y)
).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

In [15]:
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error

input_shape = (img_width, img_height, 3)
enet = tf.keras.applications.EfficientNetB0(
    include_top = False, 
    weights = 'imagenet',
    input_shape = input_shape,
    pooling = 'avg'
)
inputs = tf.keras.Input(shape = input_shape)
x = enet(inputs, training = False)
x = tf.keras.layers.Dropout(0.2)(x)
d1 = tf.keras.layers.Dense(64, activation = 'relu', kernel_initializer = 'HeUniform')
x = d1(x)
d2 = tf.keras.layers.Dense(len(targets))
outputs = d2(x)
m = tf.keras.models.Model(inputs, outputs)

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-5,
    decay_steps=5000,
    alpha=0.1
)

m.compile(
    loss = tf.keras.losses.CosineSimilarity(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),
    metrics = [tf.keras.metrics.CosineSimilarity()]
)
df_true = pd.DataFrame(
    target_proc.inverse_transform(
        df_spots.loc[df_spots['slide'] == 5, targets]
    ), index = df_spots[df_spots['slide'] == 5].index, columns = targets
)
for i in range(35):
    hist = m.fit(ds_cv_train, epochs = 1)
    df_prd = pd.DataFrame(
        target_proc.inverse_transform(m.predict(ds_valid)), index = df_spots[df_spots['slide'] == 5].index, columns = targets
    )
    print(
        df_true.apply(lambda x: spearmanr(x, df_prd.loc[x.name])[0], axis=1).mean(),
        mean_squared_error(df_true.stack(), df_prd.stack())
    )

251/251 ━━━━━━━━━━━━━━━━━━━━ 59s 86ms/step - cosine_similarity: 0.1901 - loss: -0.1901
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 348ms/step
-0.1828542392566783 1.2145490042251401
251/251 ━━━━━━━━━━━━━━━━━━━━ 22s 86ms/step - cosine_similarity: 0.6156 - loss: -0.6156
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
-0.06805868688938992 0.8448595119321661
251/251 ━━━━━━━━━━━━━━━━━━━━ 22s 86ms/step - cosine_similarity: 0.7499 - loss: -0.7499
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.13173549907767987 0.5995709950362964
251/251 ━━━━━━━━━━━━━━━━━━━━ 22s 86ms/step - cosine_similarity: 0.8234 - loss: -0.8234
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.26317380610780905 0.49243702520866844
251/251 ━━━━━━━━━━━━━━━━━━━━ 22s 87ms/step - cosine_similarity: 0.8582 - loss: -0.8582
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0.3736353077816493 0.437604775009259
251/251 ━━━━━━━━━━━━━━━━━━━━ 22s 87ms/step - cosine_similarity: 0.8784 - loss: -0.8784
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.4623778779804605 0.40529328739359477
251

# Fine Tuning

과적합을 유의해야하는 데이터셋으로 판단되고 큰 도움은 되지 않습니다.

In [16]:
"""
inputs = tf.keras.Input(shape = input_shape)
x = enet(inputs, training = True)
x = tf.keras.layers.Dropout(0.2)(x)
x = d1(x)
outputs = d2(x)
m = tf.keras.models.Model(inputs, outputs)
m.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6),  # Low learning rate
    loss=tf.keras.losses.CosineSimilarity(),
    metrics=[tf.keras.metrics.CosineSimilarity()],
)
for i in range(10):
    hist = m.fit(ds_cv_train, epochs=1)
    df_prd = pd.DataFrame(
        target_proc.inverse_transform(m.predict(ds_valid)), index = df_spots[df_spots['slide'] == 5].index, columns = targets
    )
    print(
        df_true.apply(lambda x: spearmanr(x, df_prd.loc[x.name])[0], axis=1).mean(),
        mean_squared_error(df_true.stack(), df_prd.stack())
    )
"""

"\ninputs = tf.keras.Input(shape = input_shape)\nx = enet(inputs, training = True)\nx = tf.keras.layers.Dropout(0.2)(x)\nx = d1(x)\noutputs = d2(x)\nm = tf.keras.models.Model(inputs, outputs)\nm.compile(\n    optimizer=tf.keras.optimizers.Adam(1e-6),  # Low learning rate\n    loss=tf.keras.losses.CosineSimilarity(),\n    metrics=[tf.keras.metrics.CosineSimilarity()],\n)\nfor i in range(10):\n    hist = m.fit(ds_cv_train, epochs=1)\n    df_prd = pd.DataFrame(\n        target_proc.inverse_transform(m.predict(ds_valid)), index = df_spots[df_spots['slide'] == 5].index, columns = targets\n    )\n    print(\n        df_true.apply(lambda x: spearmanr(x, df_prd.loc[x.name])[0], axis=1).mean(),\n        mean_squared_error(df_true.stack(), df_prd.stack())\n    )\n"

# Train

In [7]:
batch_size = 32
ds_train = create_tf_ds(
    df_spots.pipe(
        lambda x: pd.concat([
            x, x.sample(n = batch_size - (len(x) % batch_size))
        ])
    )
).shuffle(5000).map(
    lambda X, Y: (proc_images(X, images), Y)
).map(
    lambda X, Y: (data_augmentation(X), Y)
).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

input_shape = (img_width, img_height, 3)
enet = tf.keras.applications.EfficientNetB0(
    include_top = False, 
    weights = 'imagenet',
    input_shape = input_shape,
    pooling = 'avg'
)
inputs = tf.keras.Input(shape = input_shape)
x = enet(inputs, training = False)
x = tf.keras.layers.Dropout(0.2)(x)
d1 = tf.keras.layers.Dense(64, activation = 'relu', kernel_initializer = 'HeUniform')
x = d1(x)
d2 = tf.keras.layers.Dense(len(targets))
outputs = d2(x)
m = tf.keras.models.Model(inputs, outputs)

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=3e-6,
    decay_steps=5000,
    alpha=0.1
)

m.compile(
    loss = tf.keras.losses.CosineSimilarity(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),
    metrics = [tf.keras.metrics.CosineSimilarity()]
)
hist = m.fit(ds_train, epochs = 30)

Epoch 1/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 91s 165ms/step - cosine_similarity: -0.0657 - loss: 0.0657
Epoch 2/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 24s 75ms/step - cosine_similarity: 0.1597 - loss: -0.1597
Epoch 3/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 22s 84ms/step - cosine_similarity: 0.3310 - loss: -0.3310
Epoch 4/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - cosine_similarity: 0.4517 - loss: -0.4517
Epoch 5/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - cosine_similarity: 0.5387 - loss: -0.5387
Epoch 6/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 75ms/step - cosine_similarity: 0.6025 - loss: -0.6025
Epoch 7/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 42s 77ms/step - cosine_similarity: 0.6504 - loss: -0.6504
Epoch 8/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - cosine_similarity: 0.6870 - loss: -0.6870
Epoch 9/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 22s 86ms/step - cosine_similarity: 0.7123 - loss: -0.7123
Epoch 10/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - cosine_similarity: 0.7327 - loss: -0.7327
Epoch 11/30
261/26

In [8]:
joblib.dump(m.get_weights(), 'model/eff_b0_3.joblib')

['model/eff_b0_3.joblib']

In [9]:
joblib.dump(target_proc, 'model/target_proc_3.joblib') 

['model/target_proc_3.joblib']

In [21]:
ds_test = create_tf_ds(df_spots_test)

df_submission = pd.DataFrame(
    np.exp(
        m.predict(
            ds_test.map(lambda X: proc_images(X)).batch(32)
        )
    ), columns = targets
).reset_index().rename(columns = {'index': 'ID'})

33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step


In [22]:
df_submission

,ID,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35
0,0,0.106880,0.117950,0.103375,0.005968,0.003798,0.024129,0.035309,0.008406,0.098681,...,0.013731,0.019819,0.012259,0.018550,0.002482,0.061982,0.069548,0.009430,0.010013,0.059851
1,1,0.128045,0.116623,0.138590,0.010258,0.007435,0.050321,0.042165,0.009597,0.204422,...,0.015731,0.056274,0.016234,0.026071,0.001425,0.072266,0.080541,0.011042,0.010376,0.076468
2,2,0.099630,0.098611,0.100010,0.008563,0.018887,0.026819,0.027433,0.006014,0.137513,...,0.006670,0.036991,0.006439,0.011987,0.001200,0.062143,0.067901,0.003298,0.007506,0.039391
3,3,0.180319,0.137755,0.158754,0.006162,0.006356,0.041420,0.081333,0.030389,0.120627,...,0.032568,0.036748,0.034174,0.035257,0.004906,0.114594,0.134391,0.015988,0.024055,0.124492
4,4,0.087845,0.145078,0.084427,0.001418,0.001762,0.010620,0.054116,0.025442,0.053148,...,0.036336,0.012387,0.023168,0.025999,0.013443,0.077650,0.096791,0.019800,0.012882,0.080666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,2083,0.076367,0.124544,0.080420,0.005161,0.014594,0.023217,0.056135,0.016682,0.076528,...,0.019992,0.027192,0.016148,0.040179,0.002422,0.089227,0.089570,0.010097,0.009481,0.072841
2084,2084,0.109420,0.134732,0.117060,0.002985,0.003336,0.030402,0.056009,0.022790,0.135976,...,0.039139,0.036089,0.032287,0.035312,0.006070,0.085672,0.118656,0.019103,0.013551,0.114055
2085,2085,0.149589,0.089491,0.120247,0.006303,0.009186,0.106010,0.041300,0.011640,0.279060,...,0.021469,0.122370,0.035616,0.024045,0.000779,0.060093,0.092103,0.008761,0.014749,0.097737
2086,2086,0.088822,0.163909,0.087320,0.001273,0.002170,0.009971,0.064393,0.025293,0.044465,...,0.043730,0.010352,0.019291,0.025303,0.020673,0.084443,0.089774,0.022668,0.014188,0.087110
